# Baseline Experiment v2 - Correct Tree Geometry

Using the CORRECT tree geometry from the official getting-started notebook.

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
from shapely.strtree import STRtree
from shapely.ops import unary_union
from decimal import Decimal, getcontext
import json
import shutil

getcontext().prec = 25
scale_factor = Decimal('1e15')

# CORRECT tree geometry from official notebook
def get_correct_tree_polygon():
    """Create the CORRECT Christmas tree polygon shape."""
    trunk_w = Decimal('0.15')
    trunk_h = Decimal('0.2')
    base_w = Decimal('0.7')
    mid_w = Decimal('0.4')
    top_w = Decimal('0.25')
    tip_y = Decimal('0.8')
    tier_1_y = Decimal('0.5')
    tier_2_y = Decimal('0.25')
    base_y = Decimal('0.0')
    trunk_bottom_y = -trunk_h

    vertices = [
        (float(Decimal('0.0')), float(tip_y)),
        (float(top_w / Decimal('2')), float(tier_1_y)),
        (float(top_w / Decimal('4')), float(tier_1_y)),
        (float(mid_w / Decimal('2')), float(tier_2_y)),
        (float(mid_w / Decimal('4')), float(tier_2_y)),
        (float(base_w / Decimal('2')), float(base_y)),
        (float(trunk_w / Decimal('2')), float(base_y)),
        (float(trunk_w / Decimal('2')), float(trunk_bottom_y)),
        (float(-(trunk_w / Decimal('2'))), float(trunk_bottom_y)),
        (float(-(trunk_w / Decimal('2'))), float(base_y)),
        (float(-(base_w / Decimal('2'))), float(base_y)),
        (float(-(mid_w / Decimal('4'))), float(tier_2_y)),
        (float(-(mid_w / Decimal('2'))), float(tier_2_y)),
        (float(-(top_w / Decimal('4'))), float(tier_1_y)),
        (float(-(top_w / Decimal('2'))), float(tier_1_y)),
    ]
    return Polygon(vertices)

TREE_POLY = get_correct_tree_polygon()
print(f"Tree polygon: {len(TREE_POLY.exterior.coords)} vertices")
print(f"Bounds: {TREE_POLY.bounds}")
print(f"Area: {TREE_POLY.area:.6f}")

In [ ]:
def parse_s_value(s_val):
    """Parse a string value prefixed with 's' to float."""
    if isinstance(s_val, str) and s_val.startswith('s'):
        return float(s_val[1:])
    return float(s_val)

def load_submission(path):
    """Load submission CSV and parse values."""
    df = pd.read_csv(path)
    df['x_val'] = df['x'].apply(parse_s_value)
    df['y_val'] = df['y'].apply(parse_s_value)
    df['deg_val'] = df['deg'].apply(parse_s_value)
    df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))
    df['tree_idx'] = df['id'].apply(lambda x: int(x.split('_')[1]))
    return df

def create_tree_at_position(x, y, deg):
    """Create a tree polygon at the given position and rotation."""
    tree = rotate(TREE_POLY, deg, origin=(0, 0))
    tree = translate(tree, x, y)
    return tree

def get_bounding_box_side(polygons):
    """Get the side length of the smallest square bounding box."""
    if not polygons:
        return 0
    combined = unary_union(polygons)
    bounds = combined.bounds
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    return max(width, height)

def check_overlaps(polygons, tolerance=1e-10):
    """Check for overlapping polygons."""
    if len(polygons) <= 1:
        return []
    overlaps = []
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx > i:
                if polygons[i].intersects(polygons[idx]):
                    if not polygons[i].touches(polygons[idx]):
                        intersection = polygons[i].intersection(polygons[idx])
                        if intersection.area > tolerance:
                            overlaps.append((i, idx, intersection.area))
    return overlaps

print("Functions defined")

In [ ]:
# Load baseline submission
baseline_path = '/home/nonroot/snapshots/santa-2025/21116303805/submission/submission.csv'
df = load_submission(baseline_path)
print(f"Loaded {len(df)} rows")
print(f"N values: {df['n'].min()} to {df['n'].max()}")

In [ ]:
# Compute score for all n
scores_by_n = {}
overlap_count = 0

for n in range(1, 201):
    n_df = df[df['n'] == n]
    
    if len(n_df) != n:
        print(f"ERROR: n={n} has {len(n_df)} trees")
        continue
    
    polygons = [create_tree_at_position(row['x_val'], row['y_val'], row['deg_val']) 
                for _, row in n_df.iterrows()]
    
    overlaps = check_overlaps(polygons)
    if overlaps:
        overlap_count += 1
    
    side = get_bounding_box_side(polygons)
    score_n = (side ** 2) / n
    scores_by_n[n] = {'side': side, 'score': score_n, 'overlaps': len(overlaps)}
    
    if n <= 10 or n % 50 == 0:
        print(f"n={n:3d}: side={side:.6f}, score={score_n:.6f}, overlaps={len(overlaps)}")

print(f"\nConfigurations with overlaps: {overlap_count}")

In [ ]:
# Compute total score
total_score = sum(s['score'] for s in scores_by_n.values())
print(f"\n{'='*50}")
print(f"TOTAL SCORE: {total_score:.6f}")
print(f"{'='*50}")
print(f"\nTarget: 68.888293")
print(f"Gap: {total_score - 68.888293:.6f}")
print(f"Improvement needed: {(total_score - 68.888293) / total_score * 100:.2f}%")

In [ ]:
# Score breakdown by range
print("\nScore breakdown:")
ranges = [(1, 10), (11, 20), (21, 50), (51, 100), (101, 150), (151, 200)]
for start, end in ranges:
    range_score = sum(scores_by_n[n]['score'] for n in range(start, end+1))
    print(f"n={start:3d}-{end:3d}: {range_score:.6f}")

In [ ]:
# Copy submission to output
shutil.copy(baseline_path, '/home/submission/submission.csv')
shutil.copy(baseline_path, '/home/code/experiments/000_baseline/submission.csv')
print("Copied submission files")

# Save metrics
metrics = {
    'cv_score': total_score,
    'overlap_errors': overlap_count,
    'target': 68.888293,
    'gap': total_score - 68.888293
}

with open('/home/code/experiments/000_baseline/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\nSaved metrics: {metrics}")